In [40]:
import numpy as np
import networkx as nx
import time
#from pyvis.network import Network
import pandas as pd
import numpy as np
from collections import OrderedDict
import json
import random
import scipy

In [32]:
weight = np.array([0,1,np.inf])
rng = np.random.default_rng()

In [33]:
input_data = 'example1'
vulner = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='Vulnerabilities', index_col='id')
counter_m = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='C-Measures', index_col='id')
prevent_matrix = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='Measures-Vuln-P', index_col='id', dtype=object)
detect_matrix = pd.read_excel('.\\' + input_data + '\\data.xlsx',sheet_name='Measures-Vuln-D', index_col='id', dtype=object)

In [34]:
rng.choice(weight, size = 100, p = [0.4,0.5,0.1])

array([ 1., inf,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
        1., inf,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,
        0.,  1.,  1., inf, inf, inf,  1.,  1.,  0.,  1.,  1.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,  0., inf,  0.,
        0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,
        1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., inf,  0.,  0.,  1., inf,  1., inf,
        1.,  1.,  1., inf,  1.,  1.,  1.,  1.,  1.])

In [35]:
%%time
#G = nx.generators.random_graphs.dense_gnm_random_graph(15000,20000)

Wall time: 0 ns


In [36]:
%%time
graph_matrix = np.random.choice([0,1],size = (1500,1500), p = [0.9,0.1])

Wall time: 58.8 ms


In [37]:
%%time
G = nx.Graph(graph_matrix)

Wall time: 474 ms


In [38]:
%%time
G_s = csgraph_from_dense(graph_matrix)

Wall time: 46.9 ms


In [42]:
%%time
dist_matrix_scipy = scipy.sparse.csgraph.shortest_path(G_s)

Wall time: 2.34 s


In [49]:
%%time
dist_iter_nx = dict(nx.algorithms.shortest_paths.weighted.all_pairs_dijkstra_path_length(G))

Wall time: 6min 15s


In [56]:
dist_iter_nx[12][1245]

2

In [57]:
dist_matrix_scipy[12][1245]

2.0

In [8]:
for e in list(G.edges):
    G.edges[e]['weight'] = rng.choice(weight, size = 1, p = [0.4,0.5,0.1])

In [9]:
Link_MG = G.copy()

In [15]:
len(vulner.index)

6

In [22]:
vuln_matrix_np = np.random.choice([0,1], size = (len(vulner.index),15000), p = [0.9,0.1])

In [7]:
%%time
graph_matrix = np.random.choice([0,1],size = (20000,15000), p = [0.9,0.1])

Wall time: 6.89 s


In [ ]:
graph_matrix

In [6]:
vuln_matrix = pd.DataFrame(data=vuln_matrix_np, index = vulner.index)

array([0, 0, 0, ..., 0, 0, 0])

In [24]:
vuln_matrix = pd.DataFrame(data=vuln_matrix_np, index = vulner.index)

In [25]:
vuln_matrix

,0,1,2,3,4,5,6,7,8,9,...,14990,14991,14992,14993,14994,14995,14996,14997,14998,14999
id,,,,,,,,,,,,,,,,,,,,,
v1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
v2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
v3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
v4,0,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
v5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
v6,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
vulner.loc[vulner.AV == 'P','AV'] = -np.inf
vulner.loc[vulner.AV == 'L','AV'] = 0
vulner.loc[vulner.AV == 'A','AV'] = 1
vulner.loc[vulner.AV == 'N','AV'] = 2

vulner.loc[vulner.PR == 'N','PR'] = 0
vulner.loc[vulner.PR == 'L','PR'] = 1
vulner.loc[vulner.PR == 'H','PR'] = 2

vulner.loc[vulner['s.AV'] == 'P','s.AV'] = np.inf
vulner.loc[vulner['s.AV'] == 'L','s.AV'] = 0
vulner.loc[vulner['s.AV'] == 'A','s.AV'] = 1
vulner.loc[vulner['s.AV'] == 'N','s.AV'] = 2

vulner.loc[vulner['s.PR'] == 'N','s.PR'] = 0
vulner.loc[vulner['s.PR'] == 'L','s.PR'] = 1
vulner.loc[vulner['s.PR'] == 'H','s.PR'] = 2

In [35]:
cont_counter_m = counter_m[counter_m['Период'] == 0]
k = 1

In [36]:
for (vuln, prob) in prevent_matrix.iteritems():
    vulner.at[vuln, 'P_v'] = (1 - prob[cont_counter_m.index]).prod() * k * vulner.at[vuln,'Prob']
for (vuln, prob) in detect_matrix.iteritems():
    vulner.at[vuln, 'P_d'] = 1 - (1 - prob[cont_counter_m.index]).prod()
    

In [37]:
#CtxA
def get_asset(df, col_name, val):
    return df[df[col_name] == val]
#CtxN
def get_all_assets(df, col_name):
    return df[col_name].unique()
#Pr(a)
def get_pr_asset(df, col_name, val, pr_col_name):
    sub_df = df[df[col_name] == val]
    if len(sub_df):
        return df[df[col_name] == val][pr_col_name].max()
    else:
        return 0
#PhAdj(a)
def phys_adj(G, node):
    return [n for n in G.neighbors(node) if G.edges[node,n]['weight'] == np.inf]

In [38]:
#задаем контекст
cols = ['a_id', 'AV', 'PR', 'I']
Ctx_0 = pd.DataFrame(columns = cols)
#___________________контекст для example1___________________________________________
Ctx_0 = Ctx_0.append({'a_id' : 4, 'AV' : 0, 'PR' : 0, 'I' : 0}, ignore_index = True)
Ctx_0 = Ctx_0.append({'a_id' : 9, 'AV' : np.inf, 'PR' : 0, 'I' : 0}, ignore_index = True)
#___________________контекст для minimal___________________________________________
#Ctx_0 = Ctx_0.append({'a_id' : 0, 'AV' : 0, 'PR' : 0, 'I' : 0}, ignore_index = True)
#Ctx_0 = Ctx_0.append({'a_id' : 1, 'AV' : 0, 'PR' : 0, 'I' : 0}, ignore_index = True)

In [41]:
G_dash = nx.MultiDiGraph()

In [42]:
#инциализация графа начальным контекстом
G_dash.add_nodes_from(list(Ctx_0.index))
#nx.set_node_attributes(G_dash, Ctx_0.to_dict(orient='index'))
#вспомогательный DataFrame для более удобных выборок врешин (возможно не надо писать атрибуты в граф)
A_df = pd.DataFrame(Ctx_0)

#инициализация переменного контекста
Ctx = pd.DataFrame(Ctx_0)

In [46]:
t_begin = time.time()
#Основной цикл
while len(Ctx.index):

    #извлечь вершину из текущего контекста
    a_begin = Ctx.iloc[-1]
    Ctx = Ctx.drop([a_begin.name])
    if a_begin['AV'] == np.inf:
        a_end_list = phys_adj(Link_MG,a_begin['a_id'])
        a_end_list.append(a_begin['a_id'])
        for a_end_id in a_end_list:
            a_name = a_end_id
            pr_a = get_pr_asset(A_df, 'a_id', a_end_id, 'PR')
            v_index = list(vuln_matrix[vuln_matrix[a_name] == 1].index)
            v_sub_df = vulner.loc[v_index]
            v_sub_df = v_sub_df[v_sub_df.PR <= pr_a]
            for v_id, v_data in v_sub_df.iterrows():
                str_search = "(a_id == {a_id}) & (AV == {v_AV}) & (PR == {v_PR}) & (I == {I})".format(
                    a_id=a_end_id, 
                    v_AV = v_data['s.AV'], 
                    v_PR = v_data['s.PR'], 
                    I = v_data['Imp']
                )
                a_ex_ind = A_df.query(str_search).index
                #не найдена такая вершина в графе
                if len(a_ex_ind) == 0:
                    a_dict = {'a_id' : a_end_id, 'AV' : v_data['s.AV'], 'PR' : v_data['s.PR'], 'I' : v_data['Imp']}
                    A_df = A_df.append(a_dict, ignore_index = True)
                    a_series = A_df.iloc[-1]
                    a_ex_ind = a_series.name
                    G_dash.add_node(int(a_ex_ind))
                    if a_series['I'] == 0:
                        Ctx = Ctx.append(A_df.iloc[a_ex_ind])
                else:
                    a_ex_ind = a_ex_ind[0]
                G_dash.add_edge(int(a_begin.name), int(a_ex_ind), label = v_id)
    else:
        for a_end_id in Link_MG.nodes():
            d = 0
            if nx.algorithms.shortest_paths.generic.has_path(Link_MG, source = a_begin['a_id'], target = a_end_id):
                d = float(nx.shortest_path_length(Link_MG, source = a_begin['a_id'], target = a_end_id, weight = 'weight'))
            else:
                d = np.inf
            a_name = a_end_id
            pr_a = get_pr_asset(A_df, 'a_id', a_end_id, 'PR')
            v_index = list(vuln_matrix[vuln_matrix[a_name] == 1].index)
            if len(v_index):
                v_sub_df = vulner.loc[v_index]
                v_sub_df = v_sub_df[(v_sub_df.PR <= pr_a) & (v_sub_df.AV >= a_begin['AV'] + d)]
                for v_id, v_data in v_sub_df.iterrows():
                    str_search = "(a_id == {a_id}) & (AV == {v_AV}) & (PR == {v_PR}) & (I == {I})".format(
                        a_id=a_end_id, 
                        v_AV = v_data['s.AV'], 
                        v_PR = v_data['s.PR'], 
                        I = v_data['Imp']
                    )
                    a_ex_ind = A_df.query(str_search).index
                    #не найдена такая вершина в графе
                    if len(a_ex_ind) == 0:
                        a_dict = {'a_id' : a_end_id, 'AV' : v_data['s.AV'], 'PR' : v_data['s.PR'], 'I' : v_data['Imp']}
                        A_df = A_df.append(a_dict, ignore_index = True)
                        a_series = A_df.iloc[-1]
                        a_ex_ind = a_series.name
                        G_dash.add_node(int(a_ex_ind))
                        if a_series['I'] == 0:
                            Ctx = Ctx.append(A_df.iloc[a_ex_ind])
                    else:
                        a_ex_ind = a_ex_ind[0]
                    G_dash.add_edge(int(a_begin.name), int(a_ex_ind), label = v_id)
t_end = time.time()

OverflowError: cannot convert float infinity to integer

In [26]:
#v_sub_df = v_sub_df[(v_sub_df.PR <= pr_a) & (v_sub_df.AV >= a_begin['AV'] + d)]
#a_begin - Series
#pr_a - int
a = pd.DataFrame(columns = ['A','B','C','D'])
b = pd.Series({'A' : 0, 'B' : 10, 'C' : 2, 'D' : 6})

In [27]:
a['A'] = [1,2,5,15]
a['B'] = [10,10,10,10]
a['C'] = [5,4,3,2]
a['D'] = [5,6,12,3]

In [29]:
val_a = 11
d = [0]

In [30]:
a_new = a[(a['A'] <= val_a) & (a['B'] >= b['D'] + d)]

ValueError: ('Lengths must match to compare', (4,), (1,))

In [25]:
a_new

,A,B,C,D
0,1,10,5,5
1,2,10,4,6
2,5,10,3,12
